<a href="https://colab.research.google.com/github/pb-roshith/Sarcasm-Detection-using-Hierarchical-BERT/blob/main/Sarcasm_Detection_using_Hierarchical_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d danofer/sarcasm

Dataset URL: https://www.kaggle.com/datasets/danofer/sarcasm
License(s): copyright-authors
 89% 193M/216M [00:00<00:00, 687MB/s]
100% 216M/216M [00:00<00:00, 601MB/s]


In [4]:
from zipfile import ZipFile
dataset='/content/sarcasm.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("extracted successfully dude")

extracted successfully dude


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('/content/train-balanced-sarcasm.csv')

In [7]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [8]:
df.shape

(1010826, 10)

In [9]:
df = df[:10000]

In [11]:
df = df[['label', 'comment']]

In [12]:
df.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


In [13]:
df.shape

(10000, 2)

In [14]:
df.isna().sum()

,0
label,0
comment,1


In [15]:
df.dropna(inplace=True)

<ipython-input-15-c64f9f573c18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [16]:
df.isna().sum()

,0
label,0
comment,0


In [17]:
df.shape

(9999, 2)

In [18]:
df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

<ipython-input-18-9bc64498e39a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].str.replace(r'[^a-zA-Z\s]', '', regex=True)


In [19]:
def LowerCase(text):
  return text.lower()

In [20]:
df['comment'] = df['comment'].apply(LowerCase)

<ipython-input-20-ad5e182eebe3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comment'] = df['comment'].apply(LowerCase)


In [21]:
df.head()

,label,comment
0,0,nc and nh
1,0,you do know west teams play against west teams...
2,0,they were underdogs earlier today but since gr...
3,0,this meme isnt funny none of the new york nigg...
4,0,i could use one of those tools


In [40]:
from transformers import BertTokenizer, TFBertModel

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
def tokenizer_data(text):
  return tokenizer(
      text.tolist(),
      max_length=100,
      padding='max_length',
      truncation=True,
      return_tensors='np'
  )

In [32]:
tokenized_data = tokenizer_data(df['comment'])

In [33]:
tokenized_data

{'input_ids': array([[  101, 13316,  1998, ...,     0,     0,     0],
       [  101,  2017,  2079, ...,     0,     0,     0],
       [  101,  2027,  2020, ...,     0,     0,     0],
       ...,
       [  101,  5095,  2305, ...,     0,     0,     0],
       [  101, 29420,  2015, ...,     0,     0,     0],
       [  101,  2016, 28719, ...,     0,     0,     0]]), 'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]), 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])}

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
x = tokenized_data['input_ids']
y = df['label']

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [37]:
x_train.shape, x_test.shape

((7999, 100), (2000, 100))

In [38]:
import tensorflow as tf

In [39]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [48]:
import tensorflow as tf

class HierarchicalBERT(tf.keras.Model):
  def __init__(self, bert_model, lstm_units, cnn_filters, dense_units):
    super(HierarchicalBERT, self).__init__()
    self.bert = bert_model

    self.dense_sentence = tf.keras.layers.Dense(768, activation='relu')
    self.mean_pooling = tf.keras.layers.GlobalAveragePooling1D()
    self.bilstm_encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=lstm_units, return_sequences=True))
    self.conv = tf.keras.layers.Conv1D(filters=cnn_filters, kernel_size=3, activation='relu')
    self.pool = tf.keras.layers.GlobalMaxPool1D()
    self.dense_df = tf.keras.layers.Dense(dense_units, activation='relu')
    self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid')

  def call(self, inputs):
    bert_output = self.bert(inputs)[0]
    sentence_encoded = self.dense_sentence(bert_output)
    context_summarized = self.mean_pooling(sentence_encoded)
    context_summarized = tf.expand_dims(context_summarized, 1)
    context_encoded = self.bilstm_encoder(context_summarized)
    context_encoded_squeezed = tf.squeeze(context_encoded, axis=1)
    context_encoded_expanded = tf.expand_dims(context_encoded_squeezed, axis=-1)
    conv_output = self.conv(context_encoded_expanded)
    pool_output = self.pool(conv_output)
    dense_output = self.dense_df(pool_output)
    final_output = self.output_layer(dense_output)
    return final_output

In [49]:
model = HierarchicalBERT(bert_model, lstm_units=128, cnn_filters=64, dense_units=32)

In [50]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 78s 225ms/step - accuracy: 0.6325 - loss: 0.6615
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 78s 222ms/step - accuracy: 0.6298 - loss: 0.6602
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 222ms/step - accuracy: 0.6196 - loss: 0.6655
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 55s 220ms/step - accuracy: 0.6174 - loss: 0.6661
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 56s 222ms/step - accuracy: 0.6249 - loss: 0.6600


In [52]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Model Accuracy : {accuracy*100}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 213ms/step - accuracy: 0.6398 - loss: 0.6516
Model Accuracy : 63.749998807907104
